In [1]:
# Training script for LazyConfig models
try:
    # ignore ShapelyDeprecationWarning from fvcore
    import warnings

    from shapely.errors import ShapelyDeprecationWarning

    warnings.filterwarnings("ignore", category=ShapelyDeprecationWarning)

except:
    pass
warnings.filterwarnings("ignore", category=RuntimeWarning)
warnings.filterwarnings("ignore", category=UserWarning)

# Some basic setup:
# Setup detectron2 logger
from detectron2.utils.logger import setup_logger

setup_logger()

import gc
import os
import time

import detectron2.utils.comm as comm

# import some common libraries
import numpy as np
import torch

# import some common detectron2 utilities
from detectron2.config import LazyConfig, get_cfg
import detectron2.data as data
from detectron2.engine import (
    launch,
)

from deepdisc.data_format.augment_image import train_augs
from deepdisc.data_format.image_readers import DC2ImageReader
from deepdisc.data_format.register_data import (
    register_data_set,
)  # , register_loaded_data_set
from deepdisc.model.loaders import (
    RedshiftFlatDictMapper,
    return_test_loader,
    return_train_loader,
)
from deepdisc.model.models import (
    RedshiftPointCasROIHeads,
    RedshiftPointROIHeads,
    RedshiftPDFROIHeads,
    return_lazy_model,
)
from deepdisc.training.trainers import (
    return_evallosshook,
    return_lazy_trainer,
    return_optimizer,
    return_savehook,
    return_schedulerhook,
)
from deepdisc.utils.parse_arguments import make_training_arg_parser

In [2]:
# import importlib
# importlib.reload(rail.estimation.algos.deepdisc)

In [2]:
# from rail.estimation.algos.deepdisc import DeepDiscInformer
import rail
from rail.estimation.algos.deepdisc import *
from rail.core.data import TableHandle, JsonHandle
from rail.core.stage import RailStage

from rail.deepdisc.configs import *

In [3]:
#/home/shared/hsc/DC2/test_data/dataset_3/flattened_images_train_small.npy
cfgfile = "/home/shared/hsc/detectron2/configs/COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_1x.py"
dirpath = "./tests/deepdisc/test_data/dc2/"
output_dir = "./"
output_name = "test"

#trainfile = dirpath + "flattened_data_test.npy"
trainfile = "/home/shared/hsc/DC2/test_data/dataset_3/flattened_images_train.hdf5"
testfile = dirpath + "flattened_data_test.npy"
metadatafile = "/home/shared/hsc/DC2/test_data/dataset_3/train_metadata.json"
classes = ["object"]
numclasses = 1

In [4]:
cfg = get_lazy_config(cfgfile, 1, 1)
cfg.train.init_checkpoint

'detectron2://ImageNetPretrained/MSRA/R-50.pkl'

In [5]:
DS = RailStage.data_store
DS.__class__.allow_overwrite = True

In [6]:
from deepdisc.data_format.file_io import get_data_from_json

#testdata = np.load(trainfile, allow_pickle=True)
testdata = DS.read_file("testdata", TableHandle, trainfile)
metadata = get_data_from_json(metadatafile)

In [8]:
#testdata = np.load("./tests/deepdisc/test_data/dc2/flattened_data_test.npy")
mapper = RedshiftFlatDictMapper().map_data

In [9]:
len(testdata)

10

In [10]:
dataset_dicts = {}
dds = []
for row in testdata:
    dds.append(mapper(row))
dataset_dicts["test"] = dds

In [9]:
training = DS.add_data("training", testdata(), TableHandle) #()["images"]
testing = DS.add_data("testing", testdata, TableHandle)

metadatahandle = DS.add_data("metadata", metadata, JsonHandle)

In [28]:
import h5py

f = h5py.File(trainfile, 'r')

f.keys()

In [37]:
training

In [26]:
print(testdata)

<class 'rail.core.data.TableHandle'> /home/shared/hsc/DC2/test_data/dataset_3/flattened_images_train.hdf5, (wd)


In [10]:
deep_dict = dict(
    epochs=200,
    numclasses=1,
    batch_size=1,
    output_dir="./",
    cfgfile="/home/shared/hsc/detectron2/configs/COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_1x.py",
    output_name="test_informer",
    #hdf5_groupname="images",
    
)

In [11]:
# Inform = DeepDiscInformer.make_stage(name='Inform_DeepDISC', model='detectron2://ImageNetPretrained/MSRA/R-50.pkl', **deep_dict)
Inform = DeepDiscInformer.make_stage(
    name="Inform_DeepDISC", model="test_informer.pkl", **deep_dict
)

In [ ]:
Inform.inform(training, metadatahandle)

### Inference

In [14]:
# Estimator = DeepDiscEstimator.make_stage(name='DeepDiscEstimator',
#                                       model=Inform.get_handle('model'), **deep_dict)

Estimator = DeepDiscPDFEstimator.make_stage(
    name="DeepDiscEstimator",
    model=Inform.get_handle("model"),
    hdf5_groupname=None,
    **deep_dict,
)

In [15]:
results = Estimator.estimate(testing)

[12/01 16:58:02 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from /home/g4merz/deepdisc/Swin_test.pth ...
Processing Data
Matching objects
Inserting handle into data store.  output_DeepDiscEstimator: inprogress_output_DeepDiscEstimator.hdf5, DeepDiscEstimator


In [16]:
res = results.read()

In [17]:
res